In [17]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pickle
import pandas, numpy, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import re
import nltk
import csv
from nltk.corpus import stopwords
import numpy as np

In [18]:
def stopword_remover(text):
    leng = len(texts)
    
    final_string = ""
    all_sentences = nltk.sent_tokenize(processed_article)

    all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

    # Removing Stop Words
    from nltk.corpus import stopwords  
    for i in range(len(all_words)):  
        #print(all_words[i])
        all_words[i] = [w for w in all_words[i] if w not in stopwords.words('english')]
        final_string = final_string +" "+str(all_words[i])
    print(all_words[i])    
#     print(type(final_string))
#     print(final_string)
#     return final_string

In [19]:
import os
import json
import string

labels, texts = [], []


path = '/home/raj/UB_Stuff/CSE_635/Phase2/Invalid_Data/Windows/Windows_Invalid_json'
for file in os.listdir(path):
     xpath = os.path.join(path,file)
     if xpath.endswith('.json'):
         with open(xpath, "r+") as f:
            data = json.load(f)

         temp = data["text"] 
         
         
         if temp is not None: 
             processed_article = temp.lower() 
             processed_article = re.sub('[^a-zA-Z]', ' ', processed_article )
             processed_article = re.sub(r'\s+', ' ', processed_article)
             processed_article = ' '.join([word for word in processed_article.split() if word not in (stopwords.words('english'))])
             texts.append(processed_article)
             labels.append(data["label"])
                
counter = 1
with open('/home/raj/UB_Stuff/CSE_635/Final_Data/Thailand/Protest.csv','r') as csvinput:
    reader = csv.reader(csvinput)
    for row in reader:
        try:
            if(counter == 40) :
                break
            processed_article = row[27]
            processed_article = re.sub(r'\([^)]*\)', '', processed_article)
            processed_article = " ".join(re.findall(r"[.',a-zA-Z0-9]+", processed_article))
            processed_article = ' '.join([word for word in processed_article.split() if word not in (stopwords.words('english'))])
            ##print(processed_article)
            texts.append(processed_article)
            counter+=1
            labels.append(1)

        except Exception as e:
            print(e)
            pass

counter = 1
with open('/home/raj/UB_Stuff/CSE_635/Final_Data/Thailand/Violence.csv','r') as csvinput:
    reader = csv.reader(csvinput)
    for row in reader:
        try:
            if(counter == 40) :
                break
            processed_article = row[27]
            processed_article = re.sub(r'\([^)]*\)', '', processed_article)
            processed_article = " ".join(re.findall(r"[.',a-zA-Z0-9]+", processed_article))
            processed_article = ' '.join([word for word in processed_article.split() if word not in (stopwords.words('english'))])
            #print(processed_article)
            texts.append(processed_article)
            counter+=1
            labels.append(2)

        except Exception as e:
            print(e)
            pass
        

        
# create a dataframe using texts and lables
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [20]:
# split the dataset into training and validation datasets 
sentences_train, sentences_test, y_train, y_test = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.33, random_state=42)


In [21]:

# ngram level tf-idf 
#tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3))
filename = '/home/raj/UB_Stuff/CSE_635/Phase4/TFIDF_Vector.pickle'
tfidf_vect_ngram = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', ngram_range=(1, 2), stop_words='english')
tfidf_vect_ngram.fit(trainDF['text'])
pickle.dump(tfidf_vect_ngram, open(filename, "wb"))

X_train =  tfidf_vect_ngram.transform(sentences_train)
X_test =  tfidf_vect_ngram.transform(sentences_test)

In [22]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.941834451901566


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [23]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)
score = clf.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.9440715883668904


In [24]:
## Import the Classifier.
from sklearn.neighbors import KNeighborsClassifier
## Instantiate the model with 5 neighbors. 
knn = KNeighborsClassifier(n_neighbors=3)
## Fit the model on the training data.
knn.fit(X_train, y_train)
## See how the model performs on the test data.
knn.score(X_test, y_test)

0.9955257270693513

In [25]:
import pickle
# save the model to disk
filename = '/home/raj/UB_Stuff/CSE_635/Phase4/phasr_4_model.sav'
pickle.dump(knn, open(filename, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9955257270693513


In [26]:
input= "Arnold Schwarzenegger was part of a terrorist atttack on unarmed civilians that killed 10"
input=[input]
# create a dataframe using texts and lables
#predictDF = pandas.DataFrame()
something = tfidf_vect_ngram.transform(input)
output = knn.predict(something)
print(input)
print(output)

['Arnold Schwarzenegger was part of a terrorist atttack on unarmed civilians that killed 10']
[2]
